In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns

from clean import clean_games

In [2]:
game_df = clean_games("data/games_scraped.pickle")

In [3]:
pd.set_option("display.max_columns", 250)
pd.set_option("display.max_rows", 73)

In [4]:
mask = (game_df.game_id == 'chi-gnb-1960-12-04')
game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']].head()

,game_id,team,opp,prev_rush_yds_off,prev_rush_yds_off_opp,roll3_margin,roll3_margin_opp
2946,chi-gnb-1960-12-04,chi,gnb,221.0,118.0,9.000,6.333
7479,chi-gnb-1960-12-04,gnb,chi,118.0,221.0,6.333,9.000


In [21]:
# construct a list containing all of the columns we could use in model
indicators = ['prev_', 'roll3', 'ewma_']
key_variables = [col for col in game_df.columns if col[:5] in indicators]
key_variables.sort()

additional = ['decade', 'date', 'team', 'home', 'opp', 'margin']

for col in additional[::-1]:
    key_variables.insert(0, col)

len(key_variables)

113

In [22]:
for column_name in key_variables:
    print(column_name)

decade
date
team
home
opp
margin
ewma_first_down_off
ewma_first_down_off_opp
ewma_margin
ewma_margin_opp
ewma_pass_yds_def
ewma_pass_yds_def_opp
ewma_pass_yds_off
ewma_pass_yds_off_opp
ewma_pts_def
ewma_pts_def_opp
ewma_pts_off
ewma_pts_off_opp
ewma_result_tie
ewma_result_win
ewma_result_win_opp
ewma_result_win_opp
ewma_rush_yds_def
ewma_rush_yds_def_opp
ewma_rush_yds_off
ewma_rush_yds_off_opp
ewma_to2_def
ewma_to2_def_opp
ewma_to2_off
ewma_to2_off_opp
ewma_to_def
ewma_to_def_opp
ewma_to_off
ewma_to_off_opp
ewma_yards_def
ewma_yards_def_opp
ewma_yards_off
ewma_yards_off_opp
prev_first_down_def
prev_first_down_def_opp
prev_first_down_off
prev_first_down_off_opp
prev_losses
prev_losses_opp
prev_margin
prev_margin_opp
prev_pass_yds_def
prev_pass_yds_def_opp
prev_pass_yds_off
prev_pass_yds_off_opp
prev_pts_def
prev_pts_def_opp
prev_pts_off
prev_pts_off_opp
prev_result_tie
prev_result_tie_opp
prev_result_win
prev_result_win_opp
prev_rush_yds_def
prev_rush_yds_def_opp
prev_rush_yds_off
prev_

In [15]:
game_df.columns

Index(['team', 'year', 'team_year', 'date', 'opp', 'week_num', 'decade',
       'game_day_of_week', 'game_outcome', 'team_record',
       ...
       'ewma_yards_off_opp', 'ewma_pass_yds_off_opp', 'ewma_rush_yds_off_opp',
       'ewma_to_off_opp', 'ewma_to2_off_opp', 'ewma_yards_def_opp',
       'ewma_pass_yds_def_opp', 'ewma_rush_yds_def_opp', 'ewma_to_def_opp',
       'ewma_to2_def_opp'],
      dtype='object', length=143)

In [16]:
# print all columns
for col_name in game_df.columns:
    print(col_name)

team
year
team_year
date
opp
week_num
decade
game_day_of_week
game_outcome
team_record
pts_off
margin
pts_def
first_down_off
yards_off
pass_yds_off
rush_yds_off
to_off
first_down_def
yards_def
pass_yds_def
rush_yds_def
to_def
game_time
overtime
home
to2_off
to2_def
game_id
result_tie
result_win
wins
losses
ties
prev_wins
prev_losses
prev_ties
prev_pts_off
prev_pts_def
prev_margin
prev_first_down_off
prev_yards_off
prev_pass_yds_off
prev_rush_yds_off
prev_to_off
prev_to2_off
prev_first_down_def
prev_yards_def
prev_pass_yds_def
prev_rush_yds_def
prev_to_def
prev_to2_def
prev_result_tie
roll3_num_ties
roll3_num_wins
prev_result_win
roll3_result_win
roll3_result_tie
roll3_pts_off
roll3_pts_def
roll3_margin
roll3_first_down_off
roll3_yards_off
roll3_pass_yds_off
roll3_rush_yds_off
roll3_to_off
roll3_to2_off
roll3_yards_def
roll3_pass_yds_def
roll3_rush_yds_def
roll3_to_def
roll3_to2_def
ewma_result_win
ewma_result_tie
ewma_pts_off
ewma_pts_def
ewma_margin
ewma_first_down_off
ewma_yards_off
